In [1]:
from ftplib import FTP

import os
import shutil
import pandas as pd
import time

In [2]:
source_folder = r'R:\RawData\Massmart Data'

In [3]:
parameters = [
    {'target_dir': r'R:\RawData\Masscash\\Wholesale\ToUpload',   'keyword': 'MassCashWholesale'},
    {'target_dir': r'R:\RawData\Masscash\Retail\ToUpload',       'keyword': 'MassCashRetail'},
    {'target_dir': r'R:\RawData\Makro Updated\\ToUpload',        'keyword': 'Makro'},
    {'target_dir': r'R:\RawData\Game\ToUpload',                  'keyword': 'MassDiscounters'},
    {'target_dir': r'R:\RawData\KitKat\ToUpload',                'keyword': 'KitKat'},
    {'target_dir': r'R:\RawData\Massbuild\ToUpload',             'keyword': 'Massbuild'}
]

In [4]:
def move_files_based_on_keyword(source_dir, target_dir, keyword):
    moved_files = []

    files = os.listdir(source_dir)

    for file in files:
        if keyword in file:
            source = os.path.join(source_dir, file)
            target = os.path.join(target_dir, file)

            shutil.move(source, target)

            # Add the file and its new location to the list
            moved_files.append({'File': file, 'New Location': target})

    df = pd.DataFrame(moved_files)

    return df

In [5]:
max_inactive_time = 60 * 5

In [6]:
def monitor_files(parameters, source_folder, max_inactive_time):
    last_active_time = time.time()
    
    while True:
        new_files_detected = False

        for params in parameters:
            df = move_files_based_on_keyword(source_folder, params['target_dir'], params['keyword'])
            if not df.empty:
                last_active_time = time.time()
                print(df)
                new_files_detected = True

        inactive_time = time.time() - last_active_time

        if inactive_time >= max_inactive_time:
            print(f"No new files detected for the past {int(max_inactive_time/60)} minutes. Terminating script.")
            break

        time.sleep(10)

monitor_files(parameters, source_folder, max_inactive_time)


                                                 File  \
0      Adcock_MassCashWholesale_Delta_WE_20231022.txt   
1            Adcock_MassCashWholesale_WE_20231029.txt   
2   AfriqueSA_MassCashWholesale_Delta_WE_20231022.txt   
3         AfriqueSA_MassCashWholesale_WE_20231029.txt   
4   AlpenFoodsSA_MassCashWholesale_Delta_WE_202310...   
5      AlpenFoodsSA_MassCashWholesale_WE_20231029.txt   
6     AspenSA_MassCashWholesale_Delta_WE_20231022.txt   
7           AspenSA_MassCashWholesale_WE_20231029.txt   
8         BIC_MassCashWholesale_Delta_WE_20231022.txt   
9               BIC_MassCashWholesale_WE_20231029.txt   
10    Campari_MassCashWholesale_Delta_WE_20231022.txt   
11          Campari_MassCashWholesale_WE_20231029.txt   
12  Chill_Beverages_MassCashWholesale_Delta_WE_202...   
13  Chill_Beverages_MassCashWholesale_WE_20231029.txt   
14   ClippaSA_MassCashWholesale_Delta_WE_20231022.txt   
15         ClippaSA_MassCashWholesale_WE_20231029.txt   
16  DuracellPhoenixSA_MassCashW